In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
sys.path.insert(0, "../../scripts")
from utils import load_data, infer_L1000_features

In [2]:
data_dict = load_data(["test"], dataset = 'L1000')
# complete_features_df = data_dict["complete"]
# data_dict["complete"] = data_dict["complete"][data_dict["complete"].cell_id == "A549"].reset_index(drop = True)
# data_dict["complete"] = data_dict["complete"]


meta_features = infer_L1000_features(data_dict["test"], metadata=True)
cp_features = infer_L1000_features(data_dict["test"])

complete_features_df = data_dict["test"].reindex(cp_features, axis="columns")
complete_meta_df = data_dict["test"].reindex(meta_features, axis="columns")

In [3]:
# input_dir = '../0.download-data/data/'
# complete_meta_df = pd.read_csv(input_dir + "col_meta_level_3_REP.A_A549_only_n27837.txt", sep = "\t")

In [4]:
pd.read_csv("repurposing_info_external_moa_map_resolved.tsv",sep='\t')

,broad_sample,broad_id,pert_iname,InChIKey14,moa,target,broad_date,clinical_phase,alternative_moa,alternative_target
0,BRD-A00147595-001-01-5,BRD-A00147595,balaglitazone,IETKPTYAGKZLKY,insulin sensitizer|PPAR receptor partial agonist,PPARG,broad_id_20170327,Phase 3,NaN,NaN
1,BRD-A00218260-001-03-4,BRD-A00218260,flutrimazole,QHMWCHQXCUNUAK,sterol demethylase inhibitor,NaN,broad_id_20170327,Launched,NaN,NaN
2,BRD-A00376169-001-01-6,BRD-A00376169,KBG,FOLRUCXBTYDAQK,neprilysin inhibitor,MME,broad_id_20170327,Phase 2,NaN,NaN
3,BRD-A00546892-001-02-6,BRD-A00546892,biperiden,YSXKPIUOCJLQIE,acetylcholine receptor antagonist,CHRM1|CHRM2|CHRM3|CHRM4|CHRM5,broad_id_20170327,Launched,NaN,NaN
4,BRD-A00938334-001-01-3,BRD-A00938334,drospirenone,METQSPRSQINEEU,mineralocorticoid receptor antagonist,AR|NR3C2|PGR,broad_id_20170327,Launched,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1565,BRD-K99964838-001-11-9,BRD-K99964838,bosutinib,UBPYILGKFZZVDX,Abl kinase inhibitor|Bcr-Abl kinase inhibitor|...,ABL1|BCR|CAMK1D|CAMK2G|CDK2|FRK|FYN|HCK|LYN|MA...,broad_id_20170327,Launched,NaN,NaN
1566,BRD-M39350793-334-01-1,BRD-M39350793,NaN,XGSCYTRLSDKRRZ,NaN,NaN,broad_id_20170327,NaN,NaN,NaN
1567,BRD-M80207679-001-01-5,BRD-M80207679,NaN,QCYCDKUNJFKYEI,NaN,NaN,broad_id_20170327,NaN,NaN,NaN
1568,BRD-M89827113-001-01-5,BRD-M89827113,cromakalim,BVMDRSUIADWFLV,NaN,KCNJ8,broad_id_20170327,Phase 2,NaN,NaN


In [5]:
#obtain only the moa column from the whole dataframe
df = pd.read_csv("repurposing_info_external_moa_map_resolved.tsv",sep='\t').set_index('broad_id').reindex(index=complete_meta_df['pert_id']).reset_index().drop('pert_id',axis = 1)
moa = df.moa.dropna()
moa



0                           p21 activated kinase inhibitor
2                                    microtubule inhibitor
4                  bacterial cell wall synthesis inhibitor
5        adenosine receptor antagonist|hemoglobin antag...
6                             carbonic anhydrase inhibitor
                               ...                        
11793                                 proteasome inhibitor
11794    platelet aggregation inhibitor|structural glyc...
11796                              calcium channel blocker
11797                        serotonin receptor antagonist
11801                                       EGFR inhibitor
Name: moa, Length: 8137, dtype: object

In [6]:
pipes = moa[moa.str.contains("\|")]
unique_pipes = pipes.drop_duplicates()
no_pipes = moa[~moa.str.contains("\|")]

In [7]:
#count the number of occurrences of each moa in each moa combination
moas_occurences_all_pipes = []
for pipe in unique_pipes:
    moas = pipe.split("|")
    moas_occurences = [pipes[pipes == pipe].count()]
    for moa in moas:
        moas_occurences.append(no_pipes[no_pipes == moa].count())
    moas_occurences_all_pipes.append(moas_occurences)
moas_occurrences_all_pipes_df = pd.DataFrame(data=moas_occurences_all_pipes, index = unique_pipes.index.values.tolist(), columns=["full moa occurrence","moa1 occurrence", "moa2 occurrence","moa3 occurrence", "moa4 occurrence", "moa5 occurrence", "moa6 occurrence"])
moas_occurrences_all_pipes_df

,full moa occurrence,moa1 occurrence,moa2 occurrence,moa3 occurrence,moa4 occurrence,moa5 occurrence,moa6 occurrence
5,5,23,0,NaN,NaN,NaN,NaN
19,4,0,0,0.0,NaN,NaN,NaN
21,6,22,40,NaN,NaN,NaN,NaN
46,21,0,44,20.0,NaN,NaN,NaN
51,5,22,18,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
9501,1,9,0,NaN,NaN,NaN,NaN
9547,1,24,8,NaN,NaN,NaN,NaN
9816,1,13,0,0.0,NaN,NaN,NaN
10112,2,78,0,0.0,NaN,NaN,NaN


In [8]:
moas_occurrence_df = pd.concat([unique_pipes, moas_occurrences_all_pipes_df],axis = 1)
moas_occurrence_df = moas_occurrence_df.sort_values("full moa occurrence", ascending = False)
moas_occurrence_df.index = moas_occurrence_df['moa']
moas_occurrence_df = moas_occurrence_df.drop('moa',axis =1)
moas_occurrence_df.to_csv("moas_occurrence.tsv", sep = "\t")
moas_occurrence_df.head(10)

,full moa occurrence,moa1 occurrence,moa2 occurrence,moa3 occurrence,moa4 occurrence,moa5 occurrence,moa6 occurrence
moa,,,,,,,
NFkB pathway inhibitor|proteasome inhibitor,231,16,271,NaN,NaN,NaN,NaN
Bcr-Abl kinase inhibitor|ephrin inhibitor|KIT inhibitor|PDGFR tyrosine kinase receptor inhibitor|SRC inhibitor|tyrosine kinase inhibitor,67,8,0,0.0,2.0,22.0,38.0
dopamine receptor antagonist|serotonin receptor antagonist,45,96,75,NaN,NaN,NaN,NaN
mTOR inhibitor|PI3K inhibitor,43,82,71,NaN,NaN,NaN,NaN
cyclooxygenase inhibitor|prostanoid receptor antagonist,30,113,15,NaN,NaN,NaN,NaN
KIT inhibitor|PDGFR tyrosine kinase receptor inhibitor|VEGFR inhibitor,24,0,2,40.0,NaN,NaN,NaN
AXL kinase inhibitor|c-Met inhibitor|FLT3 inhibitor|hepatocyte growth factor receptor inhibitor|macrophage migration inhibiting factor inhibitor|tyrosine kinase inhibitor,24,4,29,20.0,21.0,0.0,38.0
Bcr-Abl kinase inhibitor|KIT inhibitor|PDGFR tyrosine kinase receptor inhibitor,24,8,0,2.0,NaN,NaN,NaN
FGFR inhibitor|PDGFR tyrosine kinase receptor inhibitor|VEGFR inhibitor,23,19,2,40.0,NaN,NaN,NaN


In [9]:
moas_occurrence_df

,full moa occurrence,moa1 occurrence,moa2 occurrence,moa3 occurrence,moa4 occurrence,moa5 occurrence,moa6 occurrence
moa,,,,,,,
NFkB pathway inhibitor|proteasome inhibitor,231,16,271,NaN,NaN,NaN,NaN
Bcr-Abl kinase inhibitor|ephrin inhibitor|KIT inhibitor|PDGFR tyrosine kinase receptor inhibitor|SRC inhibitor|tyrosine kinase inhibitor,67,8,0,0.0,2.0,22.0,38.0
dopamine receptor antagonist|serotonin receptor antagonist,45,96,75,NaN,NaN,NaN,NaN
mTOR inhibitor|PI3K inhibitor,43,82,71,NaN,NaN,NaN,NaN
cyclooxygenase inhibitor|prostanoid receptor antagonist,30,113,15,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
cytokine production inhibitor|NFkB pathway inhibitor|TP53 activator,1,0,16,0.0,NaN,NaN,NaN
CDK inhibitor|glycogen synthase kinase inhibitor,1,115,6,NaN,NaN,NaN,NaN
acetylcholine receptor agonist|benzodiazepine receptor agonist,1,29,25,NaN,NaN,NaN,NaN
